In [10]:
import os 
import cv2 
import numpy as np 
import torch 
import matplotlib.pyplot as plt 

In [11]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html


In [4]:
!git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 24992, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 24992 (delta 11), reused 18 (delta 6), pack-reused 24961
Receiving objects: 100% (24992/24992), 37.79 MiB | 12.09 MiB/s, done.
Resolving deltas: 100% (17512/17512), done.


In [5]:
!cd mmdetection; python setup.py install

running install
running bdist_egg
running egg_info
creating mmdet.egg-info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mmdet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mmdet
copying mmdet/version.py -> build/lib/mmdet
copying mmdet/__init__.py -> build/lib/mmdet
creating build/lib/mmdet/models
copying mmdet/models/__init__.py -> build/lib/mmdet/models
copying mmdet/models/builder.py -> build/lib/mmdet/models
creating build/lib/mmdet/utils
copying mmdet/utils/contextmanagers.py -> build/lib/mmdet/utils
copying mmdet/utils/__init__.py -> build/lib/mmdet/utils
copying

In [13]:
from mmdet.apis import init_detector, inference_detector 
import mmcv

In [2]:
!cd mmdetection; mkdir checkpoints

In [3]:
!wget -O /content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

--2022-07-04 05:40:14--  http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.74.197.77
Connecting to download.openmmlab.com (download.openmmlab.com)|47.74.197.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167287506 (160M) [application/octet-stream]
Saving to: ‘/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’

/content/mmdetectio 100%[===================>] 159.54M  26.9MB/s    in 6.9s    

2022-07-04 05:40:21 (23.2 MB/s) - ‘/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’ saved [167287506/167287506]



In [4]:
!ls -lia /content/mmdetection/checkpoints

total 163376
3407965 drwxr-xr-x  2 root root      4096 Jul  4 05:40 .
3276802 drwxr-xr-x 19 root root      4096 Jul  4 05:40 ..
3407966 -rw-r--r--  1 root root 167287506 Nov  2  2021 faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


In [9]:
config_file = '/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = '/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

load checkpoint from local path: /content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


In [7]:
!mkdir data

In [56]:
!wget -O /content/data/John_Wick_small.mp4 https://github.com/chulminkw/DLCV/blob/master/data/video/John_Wick_small.mp4?raw=true

--2022-07-04 07:33:14--  https://github.com/chulminkw/DLCV/blob/master/data/video/John_Wick_small.mp4?raw=true
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/chulminkw/DLCV/raw/master/data/video/John_Wick_small.mp4 [following]
--2022-07-04 07:33:15--  https://github.com/chulminkw/DLCV/raw/master/data/video/John_Wick_small.mp4
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chulminkw/DLCV/master/data/video/John_Wick_small.mp4 [following]
--2022-07-04 07:33:15--  https://raw.githubusercontent.com/chulminkw/DLCV/master/data/video/John_Wick_small.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.

In [57]:
import cv2

video_reader = mmcv.VideoReader("/content/data/John_Wick_small.mp4")
video_writer = None 
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

video_writer = cv2.VideoWriter("/content/data/John_Wick_small_out1.mp4", fourcc, video_reader.fps, (video_reader.width, video_reader.height))

for frame in mmcv.track_iter_progress(video_reader):
    result = inference_detector(model, frame)
    frame = model.show_result(frame, result, score_thr = 0.4)
    
    video_writer.write(frame)
 
if video_writer:
    video_writer.release()


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 58/58, 3.5 task/s, elapsed: 17s, ETA:     0s


In [58]:
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorbike',4:'aeroplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',10:'fire hydrant',
                        11:'stop sign',12:'parking meter',13:'bench',14:'bird',15:'cat',16:'dog',17:'horse',18:'sheep',19:'cow',20:'elephant',
                        21:'bear',22:'zebra',23:'giraffe',24:'backpack',25:'umbrella',26:'handbag',27:'tie',28:'suitcase',29:'frisbee',30:'skis',
                        31:'snowboard',32:'sports ball',33:'kite',34:'baseball bat',35:'baseball glove',36:'skateboard',37:'surfboard',38:'tennis racket',39:'bottle',40:'wine glass',
                        41:'cup',42:'fork',43:'knife',44:'spoon',45:'bowl',46:'banana',47:'apple',48:'sandwich',49:'orange',50:'broccoli',
                        51:'carrot',52:'hot dog',53:'pizza',54:'donut',55:'cake',56:'chair',57:'sofa',58:'pottedplant',59:'bed',60:'diningtable',
                        61:'toilet',62:'tvmonitor',63:'laptop',64:'mouse',65:'remote',66:'keyboard',67:'cell phone',68:'microwave',69:'oven',70:'toaster',
                        71:'sink',72:'refrigerator',73:'book',74:'clock',75:'vase',76:'scissors',77:'teddy bear',78:'hair drier',79:'toothbrush' }
                          

In [59]:
def get_detected_image(model, image_array, score_threshold=0.3, is_print=True):
    draw_image  = image_array.copy()

    bbox_color = (0, 255, 0)
    text_color = (0, 0, 255)

    results = inference_detector(model, image_array)

    for result_ind, result in enumerate(results):

        if len(result) == 0 :
            continue 
        result_filtered = result[np.where(result[:, 4]> score_threshold)]

        for i in range(len(result_filtered)):
            left = int(result_filtered[i, 0])
            top = int(result_filtered[i, 1])
            right = int(result_filtered[i, 2])
            bottom = int(result_filtered[i, 3])
            caption = "{} : {:.4f}".format(labels_to_names_seq[result_ind], result_filtered[i, 4])
            cv2.rectangle(draw_image, (left, top), (right, bottom), color=bbox_color, thickness=2)
            cv2.putText(draw_image, caption, (int(left), int(top-7)), cv2.FONT_HERSHEY_SIMPLEX, 0.37, text_color, thickness=1)
            if is_print:
                print(caption)
    
    return draw_image 

In [64]:
import time 
def do_detected_video(model, input_path, output_path, score_threshold, do_print=True):
    capture = cv2.VideoCapture(input_path)
    
    codec =  cv2.VideoWriter_fourcc(*"XVID")
    video_size = (round(capture.get(cv2.CAP_PROP_FRAME_WIDTH)), round(capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    video_fps = capture.get(cv2.CAP_PROP_FPS)

    video_writer = cv2.VideoWriter(output_path, codec, video_fps, video_size)
    
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"total frame count {frame_count}")

    btime = time.time()
    while True:
        has_frame, image_frame = capture.read()

        if not has_frame :
            print("there are no more frames to process")
            break
        stime = time.time()
        image_frame = get_detected_image(model, image_frame, score_threshold=score_threshold, is_print = False)
        
        if do_print:
            print(f"execution time per frame{round(time.time()-stime, 4)}")
        
        video_writer.write(image_frame)

    video_writer.release()
    capture.release()

    print(f"Last detection completion time {round(time.time()- btime, 4)}")

In [65]:
do_detected_video(model, '/content/data/John_Wick_small.mp4', '/content/data/John_Wick_small_out2.mp4', score_threshold=0.4, do_print=True)

total frame count 58
execution time per frame0.1626
execution time per frame0.1302
execution time per frame0.1187
execution time per frame0.113
execution time per frame0.1143
execution time per frame0.113
execution time per frame0.1144
execution time per frame0.1147
execution time per frame0.1133
execution time per frame0.1154
execution time per frame0.1165
execution time per frame0.1146
execution time per frame0.1148
execution time per frame0.1134
execution time per frame0.1146
execution time per frame0.1153
execution time per frame0.114
execution time per frame0.1143
execution time per frame0.1133
execution time per frame0.1147
execution time per frame0.128
execution time per frame0.1131
execution time per frame0.1155
execution time per frame0.115
execution time per frame0.1174
execution time per frame0.1142
execution time per frame0.1141
execution time per frame0.1149
execution time per frame0.1144
execution time per frame0.1166
execution time per frame0.1148
execution time per fram